In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
x=611080.3983777018
y=472820.18991934415

In [ ]:
import re
import service_api
from models import Ortofotomapa


URL = "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/SkorowidzeWgAktualnosci?"
c = re.compile("\{{1}.*\}{1}")


def getOrtoListbyPoint1992(x,y):
    """Zwraca listę dostępnych do pobrania ortofotomap na podstawie
    zapytania GetFeatureInfo z usługi WMS"""
    LAYERS = [
        'SkorowidzeOrtofotomapyZasiegiStarsze',
        'SkorowidzeOrtofotomapyStarsze',
        'SkorowidzeOrtofotomapyZasiegi2021',
        'SkorowidzeOrtofotomapy2021',
        'SkorowidzeOrtofotomapyZasiegi2022',
        'SkorowidzeOrtofotomapy2022',
        'SkorowidzeOrtofotomapyZasiegi2020',
        'SkorowidzeOrtofotomapy2020'
    ]
    PARAMS = {
        'SERVICE': 'WMS',
        'request': 'GetFeatureInfo',
        'version': '1.3.0',
        'layers': ','.join(LAYERS),
        'styles': '',
        'crs': 'EPSG:2180',
        'bbox': '%f,%f,%f,%f' % (y-500, x-500, y+500, x+500),
        'width': '101',
        'height': '101',
        'format': 'image/png',
        'transparent': 'true',
        'query_layers': ','.join(LAYERS),
        'i': '50',
        'j': '50',
        'INFO_FORMAT': 'text/html'
    }
    resp = service_api.getRequest(params=PARAMS, url=URL)
#     print(resp)
    if resp[0]:
        ortos = c.findall(resp[1])
        ortofotomapaList = []
        for orto in ortos:
            element = orto.strip("{").strip("}").split(',')
            params = {}
            for el in element:
                item = el.strip().split(':')
                val = item[1].strip('"')
                if len(item) > 2:
                    val = ":".join(item[1:]).strip('"')
                params[item[0]] = val
#             print(params)
#             ortofotomapa = Ortofotomapa(**params)
#             ortofotomapaList.append(ortofotomapa)
            ortofotomapaList.append(params)
        return ortofotomapaList
    else:
        return []


In [ ]:
res = getOrtoListbyPoint1992(x,y)

In [ ]:
res

In [ ]:
for i in range(10):
    res = getOrtoListbyPoint1992(x+500*i,y)
    for r in res:
        if int(r['aktualnoscRok'])>=2020:
            print(r['wielkoscPiksela'])
            print(r['zrodloDanych'])
            print(r['kolor'])
            print(r['aktualnosc'])
            print(r['calyArkuszWyeplnionyTrescia'])
            print(r['ukladWspolrzednych'])
            print(r['url'])
            print(r['godlo'])
            print('\n')

In [ ]:
dir(res[0])

# geo -> get all points in PL

In [ ]:
import geopandas as gpd

In [ ]:
from shapely.geometry import Polygon,Point

In [ ]:
gpd.datasets.available

In [ ]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
world.head()

In [ ]:
poland = world.loc[world['name'] == 'Poland']
poland.plot()

In [ ]:
pl = Polygon(poland.geometry.iloc[0])
pl

In [ ]:
pl.exterior

In [ ]:
[len(_) for _ in pl.exterior.xy]

In [ ]:
xys = np.array(pl.exterior.xy)
xys.shape

In [ ]:
plt.scatter(xys[0,:],xys[1,:])

In [ ]:
pl_minx, pl_maxx, pl_miny, pl_maxy = np.min(xys[0]), np.max(xys[0]), np.min(xys[1]), np.max(xys[1])
pl_minx, pl_maxx, pl_miny, pl_maxy

In [ ]:
plt.scatter(xys[0,:],xys[1,:])
plt.plot((pl_minx, pl_maxx,pl_maxx,pl_minx,pl_minx),(pl_miny,pl_miny, pl_maxy, pl_maxy,pl_miny),'r')

In [ ]:
pl.contains(Point(20,52))

In [ ]:
pl.contains(Point(20,49))

# WGS84 (epsg:4326) -> PL1992 (EPSG:2180)

In [ ]:
from pyproj import Proj, transform

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:4326", 'EPSG:2180')
transformer.transform(52,20)

In [ ]:
xdiff = 0.025
ydiff = 0.015
print(np.array(transformer.transform(52,20)) - np.array(transformer.transform(52-xdiff,20-ydiff)))

xs = np.arange(pl_minx, pl_maxx, xdiff)
ys = np.arange(pl_miny, pl_maxy, ydiff)
xs.shape, ys.shape

In [ ]:
cart = np.transpose([np.tile(xs, len(ys)), np.repeat(ys, len(xs))])
cart.shape

In [ ]:
wgs84s = []
for x,y in tqdm(cart):
    if pl.contains(Point(x,y)):
        wgs84s.append((x,y))
wgs84s = np.array(wgs84s)
wgs84s.shape

In [ ]:
plt.figure(figsize=(20,20))
plt.scatter(wgs84s[:,0],wgs84s[:,1])

# get metadata

In [ ]:
import time
from random import shuffle

In [ ]:
wgs84s = wgs84s.tolist()
shuffle(wgs84s)

In [ ]:
def wrapper(x,y):
    xx,yy = transformer.transform(y, x)
    res = getOrtoListbyPoint1992(xx,yy)
    return x,y,res

In [ ]:
import concurrent.futures

In [ ]:
metas = dict()
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for xwgs,ywgs in tqdm(wgs84s):
        futures.append(executor.submit(wrapper, x=xwgs,y=ywgs))
    for future in tqdm(concurrent.futures.as_completed(futures)):
        x,y,res = future.result()
        metas[(x,y)] = res

In [ ]:
# printed=False
# for xwgs,ywgs in tqdm(wgs84s):
#     if (xwgs,ywgs) in metas:
# #         print(xwgs,ywgs,'skipped')
#         continue
#     x,y = transformer.transform(ywgs, xwgs)
#     res = getOrtoListbyPoint1992(x,y)
#     metas[(xwgs,ywgs)] = res
#     if not printed and len(res)>0:
#         print(res)
#         printed = True
#     time.sleep(0.001)
# len(metas)

In [ ]:
len(metas),len(wgs84s)

# VIZ

In [ ]:
def decode_godlo(godlo):
    g = godlo.split('-')
    mn = g[0]=='M'
    _34 = g[1]=='34'
    _144 = int(g[2])-1
    ABCD = {'A':(0,0),
            'B':(1,0),
            'C':(0,1),
            'D':(1,1)}[g[3]]
    abcd = {'a':(0,0),
            'b':(1,0),
            'c':(0,1),
            'd':(1,1)}[g[4]]
    _1 =  {'1':(0,0),
            '2':(1,0),
            '3':(0,1),
            '4':(1,1)}[g[5]]
    _2 =  {'1':(0,0),
            '2':(1,0),
            '3':(0,1),
            '4':(1,1)}[g[6]]
    
    y12 = _144//12
    x12 = _144 - 12*y12
    
    xx = 16*12*_34  + x12 * 16 + ABCD[0]*8 + abcd[0]*4 + _1[0]*2 + _2[0]
    yy = 16*12*mn + y12 * 16 + ABCD[1]*8 + abcd[1]*4 + _1[1]*2 + _2[1]
    
    
    return xx,yy
    
    


In [ ]:
a = np.zeros((16*12*2,16*12*2))
for m in metas.values():
    for mm in m:
        if int(mm['aktualnoscRok'])>2018 and mm['ukladWspolrzednych']=='PL-1992':
            x,y = decode_godlo(mm['godlo'])
            a[y,x] += 1
plt.figure(figsize=(10,10))
plt.imshow(a,vmin=0,vmax=a.max(), cmap='gray');

In [ ]:
mm

In [ ]:
metas

In [ ]:
godlos=dict()
for m in metas.values():
    for mm in m:
        if int(mm['aktualnoscRok'])>2018 and mm['ukladWspolrzednych']=='PL-1992' and mm['kolor']=='RGB':
            if mm['godlo'] not in godlos:
                godlos[mm['godlo']] = mm
                continue
            if int(godlos[mm['godlo']]['aktualnoscRok']) < int(mm['aktualnoscRok']):
                godlos[mm['godlo']] = mm

In [ ]:
len(godlos)

In [ ]:
for g in godlos:
    print(godlos[g])
    break

In [ ]:
import cv2

In [ ]:
cv2.imwrite('render.png',a)

# the download!

In [ ]:
import os
import urllib.request


In [ ]:
download_dir = 'Z:\ortofotomapa\images'

In [ ]:
for g in tqdm(list(godlos.keys())):
    fname = download_dir + '/' + godlos[g]['url'].split('/')[-1]
    if not os.path.exists(fname):
        try:
            urllib.request.urlretrieve(godlos[g]['url'], fname)
        except Exception as e:
            print(e)
            print(fname)
            os.remove(fname)
        time.sleep(0.1)


In [ ]:
fname

In [ ]:
def download_wrapper(url, filename):
    try:
#         print(url)
        urllib.request.urlretrieve(url, filename)
        time.sleep(5)
        return (True, filename)
    except Exception as e:
        print(e)
        print(filename)
        time.sleep(60)
        if os.path.exists(filename):
            os.remove(filename)
        return (False, filename)

download_results = []
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
for chunk in tqdm(list(chunks(list(godlos.keys()), 10)),desc='Chunks'):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for g in chunk:
            fname = download_dir + '/' + godlos[g]['url'].split('/')[-1]
            if not os.path.exists(fname):
                futures.append(executor.submit(download_wrapper, url=godlos[g]['url'], filename=fname))
        if len(futures)>0:
            print("tasks:",len(futures))
            for future in tqdm(concurrent.futures.as_completed(futures)):
                download_results.append(future.result())
            time.sleep(10)


In [ ]:
# !dir 
#Z:\ortofotomapa\images\75161_1072048_N-34-139-C-d-1-2.tif
